<a href="https://colab.research.google.com/github/KouserImam/Inverse-Compton/blob/develop/Inverse_compton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inverse Compton
The goal of this notebook is to demonstrate Inverse Compton Scattering in AGN.

Begin by installing and importing the necessary python libraries

In [ ]:
import sys
import numpy as np
from ipywidgets import interact, interactive
import ipywidgets as widgets

!{sys.executable} -m pip install --quiet matplotlib

import matplotlib.pyplot as plt

# Types of inverse compton scattering

Define some routines for the notebook.  Create an array.

Now plot a line with variable slope.

In [ ]:

interact(plot_with_slope, m = widgets.BoundedFloatText(value=1, min=0, max = 10, step=0.1,
                                                       descriptions='Slope:', disabled=False))

interactive(children=(BoundedFloatText(value=1.0, description='m', max=10.0, step=0.1), Output()), _dom_classe…

<function __main__.plot_with_slope(m)>